In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import sys

import numpy as np
import pandas as pd
from preprocessing import (read_dcm_sequence, check_sequence, make_3d_array,
                                  resample, make_lungs_mask, extract_lungs)
from plot import show_slices
from file import makedir
from keras.models import load_model
from model import LungNet
from keras.optimizers import Adam
import os
import sys
import csv
from datagen import GeneratorFactory


Using TensorFlow backend.


In [3]:
path = 'C:\\Users\\s57kumar\\Desktop\\inputimagedata\\input\\LIDC-IDRI-0014\\01-01-2000-38612\\3000562-07402'

new_spacing = (1, 1, 1)  # (2, 2, 2)
kernel_size = 5  # 3
slice_drop_prob = 0.005
save_png = False
template = "An exception of type {0} occurred. Arguments:\n{1!r}"
PatientID=0
array_lungs = None

try:
    # Get slice sequence
    patient_id = 12345
    dcm_seq = read_dcm_sequence(patient_id, path)

    PatientID= dcm_seq[0].PatientID
    dcm_seq = check_sequence(dcm_seq)

    # Stack into 3D array in Hounsfield Units
    array, spacing = make_3d_array(dcm_seq)

    # Resample
    array_resampled = resample(array, spacing, new_spacing)

    # Extract lungs
    mask_lungs, thres = make_lungs_mask(array_resampled, kernel_size=kernel_size)
    array_lungs, box = extract_lungs(array_resampled, mask_lungs, slice_drop_prob)
    #print(array_lungs)
    npz_filename = 'temp.npz'
    np.savez_compressed(npz_filename, array_lungs=array_lungs)
except Exception as ex:
    message = template.format(type(ex).__name__, ex.args)
    print('fail', patient_id, message)
    #continue
else:
    print(
        'success',
        patient_id, thres, box,
        array.shape, array_resampled.shape, array_lungs.shape
        )

success 12345 -423 ((25, 272), (68, 269), (29, 307)) (114, 512, 512) (283, 340, 340) (247, 201, 278)


In [4]:
# load the trained convolutional neural network
sys.path.append('../')

nb_filters_per_layer = (64, 128, 256)
kernel_size = (3, 3, 3)
padding = 'same'
batch_normalization = False
spp_nb_bins_per_level = (1, 2, 4)
n_dense = (1024, 1024)
dropout_rate = None
learning_rate = 1e-7
optimizer = Adam(lr=learning_rate)
es_patience = 10
checkpoints_path =  'output_model'
import time
time_string = time.strftime('%Y%m%d_%H%M%S')
model_path = os.path.join(checkpoints_path, time_string)
print("[INFO] loading network...")
lungnet = LungNet(nb_filters_per_layer=nb_filters_per_layer, kernel_size=kernel_size, padding=padding,
                  batch_normalization=batch_normalization,spp_nb_bins_per_level=spp_nb_bins_per_level, 
                  n_dense=n_dense, dropout_rate=dropout_rate,optimizer=optimizer, es_patience=es_patience,
                 model_path=model_path)
    #prob = lungnet.predict(x)

#
    

Instructions for updating:
Colocations handled automatically by placer.
[INFO] loading network...


In [5]:
freeze = ['encoder_conv_1-2_0', 'encoder_conv_2-1_0','encoder_conv_1-2_1', 'encoder_conv_2-1_1']
lungnet.build_model(freeze=freeze)

encoder_conv_1-2_0 is set to 0
encoder_conv_2-1_0 is set to 0
encoder_conv_1-2_1 is set to 0
encoder_conv_2-1_1 is set to 0


In [6]:
lungnet.load_weights_from_file("output_model\\weights.15-0.652332.hdf5")

Loaded weights from file.


In [7]:
def predict_classes(y_scores, t):
    return ['Cancer' if y >= t else 'No Cancer' for y in y_scores]

In [8]:
if(array_lungs is not None):
    val_gen_factory = GeneratorFactory(volume_resize_factor=0.3, random_rotation=False, random_offset_range=None)
    test_gen = val_gen_factory.build_input_predict('temp.npz')
    pred_data=lungnet.predict(test_gen)
    predlabel = (predict_classes(pred_data,0.65))
    #print(predlabel)
    print("Patient Id " + str(PatientID) + " has " +  str(predlabel) + " with " + str(( pred_data * 100 )) + "% probability ")

Patient Id LIDC-IDRI-0014 has ['Cancer'] with [[79.39453]]% probability 
